<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Create Issue

**Tags:** #github #naas #naas_driver #command

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates an issue on a GitHub repository.

## Input

### Import libraries

In [ ]:
import naas
import requests
import json

### Setup variables
- `title`: My New Issue
- `description`: My Description
- `repo_url`: https://github.com/jupyter-naas/awesome-notebooks
- `token`: gitwxxxxxxxxxxxxxxx
- `body`: This variable stores the body to be send by the webhook.

In [ ]:
# Mandatory
title = "My New Issue"
description = "My Description"
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
token = None

# Optional
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    title = body.get("title").replace("\n", "")
    description = body.get("description")
    repo_url = body.get("repo_url")
    token = body.get("token")

## Model

### Create New GitHub Issue

In [ ]:
def create_new_github_issue(
    repo_url=None,
    title=None,
    description=None,
    token=None
):  
    # Init
    status = "ok"
    message = f"✅ Issue has been created, here is the link: [URL]"
    
    # Check if GitHub connection is set
    if not token and not naas.secret.get('GITHUB_TOKEN'):
        message = f"❌ Unable to Connect to GitHub. Issue not created: '{title}'"
        return status, message 
    elif not token and naas.secret.get('GITHUB_TOKEN'):
        token = naas.secret.get('GITHUB_TOKEN')
    
    # Create Issue
    try:
        # Repo name
        repo_name = repo_url.split("https://github.com/")[-1]

        # Requests
        url = f"https://api.github.com/repos/{repo_name}/issues"
        data = {}
        if title:
            data["title"] = title
        if description:
            data["body"] = description
        if len(data) > 0:
            headers = {'Authorization': f'token {token}'}
            response = requests.post(url, data=json.dumps(data), headers=headers)
            github_issue = response.json()
            url = f"{repo_url}/issues/{github_issue.get('number')}"
            message = message.replace("[URL]", url)
        else:
            message = f"❌ Nothing to create, payload is empty: {data}."
    except Exception as e:
        status = "ko"
        message = e
        print(e)
    return status, message

status, message = create_new_github_issue(
    repo_url,
    title,
    description,
    token
)
print("Status:", status)
print("Message:", message)

## Output

### Create or update Webhook

In [ ]:
naas.webhook.add()

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)